In [1]:
import pandas as pd
import numpy as np
import json 
from unidecode import unidecode
from textblob import TextBlob

## Lendo Dados

In [2]:
js = np.array(json.load(open('../hackathon/opinioes.json')))
df_opinioes = pd.DataFrame.from_dict(js)
df = pd.read_csv('../hackathon/notas.csv',sep=';')

In [8]:
len(df)

79935

In [3]:
df_opinioes.head()

,0,1,2,3,4,5,6,7,8,9
0,e2b9dc08,"""MELHOR POPULAR DO MERCADO ATÉ 2012""",Chevrolet Celta LT 1.0 2011/2012,João - Brasília DF,Dono há 6 anos - 35.200 kmCarro anterior: Fiat...,"Prós:ECONOMIA DE COMBUSTÍVEL, PEÇAS BARATAS, R...",Contras:UM POUCO BAIXO.. AS VEZES QUANDO ESTÁ ...,Defeitos apresentados:NENHUM,Opinião Geral:O CARRO É 10 ESTOU COM ELE A 7 A...,09/03/2018 17:40:00
1,3b9dcf63,"""Ótimo carro a um custo baixo!""",Chevrolet Celta LT 1.0 2015/2015,Hugo - São Pedro SP,Dono há 3 anos - 59.000 kmCarro anterior: Chev...,"Prós:Motor potente pra um carro 1.0, sem deixa...",Contras:Saiu de linha,Defeitos apresentados:Nenhum,Opinião Geral:Este é o segundo celta que compr...,02/03/2018 21:27:00
2,9f62a709,"""Excelente!""",Chevrolet Celta Life 1.0 2004/2005,Ivan - Salvador BA,Dono há menos de 1 ano - 80.000 kmCarro anteri...,"Prós:Tamanho[cabe em qualquer vaga],excelente ...",Contras:Nao tem. Levando em conta que o carro ...,Defeitos apresentados:Até agora nenhum. Já fiz...,"Opinião Geral:Recomendo: carro valente, relati...",27/02/2018 23:14:00
3,0e6c8d29,"""Rapido e economico""",Chevrolet Celta LT 1.0 2012/2013,Gabriel - Porto Alegre RS,Dono há 3 anos - 89.000 kmCarro anterior: Volk...,"Prós:Carro extremamente confiavel, so fazer a ...",Contras:Acabamento do carro é RUSTICO bem mal ...,Defeitos apresentados:nada o carro é valente,Opinião Geral:Ja passei ate por trilhas onde c...,18/02/2018 12:32:00
4,3c958631,"""Carro honesto.""",Chevrolet Celta Life 1.0 2007/2008,Deleon - Brasília DF,Dono há 7 anos - 150.247 kmCarro anterior: Vol...,"Prós:Desempenho[para 1.0], manutenção barata, ...","Contras:Meio duro, pouco espaço. Motor grita m...",Defeitos apresentados:Bomba dágua.,Opinião Geral:Carro honesto. Para primeiro vei...,04/02/2018 21:19:00


## Analise de sentimentos

In [93]:
%%time
js_copy = js.copy()

column = 1 #1,5,6,7,8
opinioes = list()
for i in range(js_copy.shape[0]):
    #========================
    #Ajustando Dados
    
    text = js_copy[i][column]
    if column == 1:   #remove aspas
        text = text[1:-1]
    elif column == 5: #remove 'Pros:'
        text = text[5:]
    elif column == 6: #remove 'Contras:'
        text = text[8:]
    elif column == 7: #remove 'Defeitos apresentados:'
        text = text[22:]
    elif column == 8: #remove 'Opinião Geral:'
        text = text[14:]
    
    #========================
    #Analise de sentimentos
    
    try:
        #tentando analisar direto
        text = TextBlob(text)
        text = TextBlob(str(text.translate(from_lang='pt',to='en')))
        opinioes.append([js_copy[i][column],str(text),text.sentiment.polarity,text.sentiment.subjectivity])
        print(opinioes[-1])
    except:
        try:
            #removendo acentos para tentar analisar
            text = TextBlob(unidecode(text))
            text = TextBlob(str(text.translate(from_lang='pt',to='en')))
            opinioes.append([js_copy[i][column],str(text),text.sentiment.polarity,text.sentiment.subjectivity])
            print(opinioes[-1])
        except:
            opinioes.append([js_copy[i][column],str(text),None,None])
            print('ERRO: ',js_copy[i][j])   


[u'"MELHOR POPULAR DO MERCADO AT\xc9 2012"', 'BEST POPULAR OF THE MARKET UNTIL 2012', 0.8, 0.6]
[u'"\xd3timo carro a um custo baixo!"', 'Great car at a low cost!', 0.4, 0.525]
[u'"Excelente!"', 'Great!', 1.0, 0.75]
[u'"Rapido e economico"', 'Fast and economical', 0.25, 0.75]
[u'"Carro honesto."', 'Honest car.', 0.6, 0.9]
[u'"Compacto mas valente"', 'Compact but brave', 0.8, 1.0]
CPU times: user 95.5 ms, sys: 205 ms, total: 301 ms
Wall time: 1.29 s


## Salvando 

In [83]:
df_op = pd.DataFrame(opinioes,columns=['id','text','polarity','subjectivity'])
df_op.to_csv('sent_'+str(column)+'.csv',index=False)